# Modèle pre-entrainer XCEPTION : classification binaire COVID / SAIN (NORMAL) par extraction de features

In [7]:
# Chargement des modules necessaire :

import os

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras import applications
from tensorflow.keras.models import Model

In [11]:
# On declare les chemins vers les donnees :

trainDir = 'Data/TRAIN'
validationDir = 'Data/TEST'
        
# On declare les dimensions pour les images (224,224) :

img_width, img_height = 224, 224

# On liste et stocke les chemins des images :

ImageTRAINCOVID = os.listdir(trainDir + '/COVID')
ImageTRAINNORMAL = os.listdir(trainDir + '/NORMAL')

ImageTESTCOVID = os.listdir(validationDir + '/COVID')
ImageTESTNORMAL = os.listdir(validationDir + '/NORMAL')

# On affiche le nombre d'image trouve :

print('Il y a ' + str(len(ImageTRAINCOVID)) + ' images d\'entrainement de patient covid.') 
print('Il y a ' + str(len(ImageTRAINNORMAL)) + ' images d\'entrainement de patient non-covid.') 
print('Il y a ' + str(len(ImageTESTCOVID)) + ' images test de patient covid.') 
print('Il y a ' + str(len(ImageTESTNORMAL)) + ' images test de patient non-covid.') 

Il y a 401 images d'entrainement de patient covid.
Il y a 401 images d'entrainement de patient non-covid.
Il y a 102 images test de patient covid.
Il y a 102 images test de patient non-covid.


In [12]:
# Visualisation de 6 exemples en 2 lignes et 3 colonnes pour chaque classe :




In [14]:
# Preprocessing :

# On rescale les images :
datagen = ImageDataGenerator(rescale=1./255)

# On definit la batch size :
batch_size = 32

# On prepare les tableaux de donnees depuis les images :

train_generator_bottleneck = datagen.flow_from_directory(
        trainDir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validationDir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)


Found 802 images belonging to 2 classes.
Found 204 images belonging to 2 classes.


In [16]:
# Chargement de XCEPTION sans la partie fully-connected avec le reseau convolutif :

model_XCEPTION = applications.Xception(include_top=False, weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
83689472/83683744 [==============================] - 121s 1us/step


In [17]:
# On utilse le model XCEPTION pour extraire les features de nos images 
# (on fait recupere la sortie du reseau convolutionnel) :

train_features = model_XCEPTION.predict_generator(train_generator_bottleneck, 802 // batch_size)

np.save(open('models/trainFeatures.npy', 'wb'), train_features)

validation_features = model_XCEPTION.predict_generator(validation_generator_bottleneck, 204 // batch_size)

# L'opération étant longue on enregistre les features obtenus :

np.save(open('models/validationFeatures.npy', 'wb'), validation_features)

In [ ]:
# Si l'operation à deja été effectuer on charge les features :

train_features = np.load(open('models/trainFeatures.npy', 'rb'))

validation_features = np.load(open('models/validationFeatures.npy', 'rb'))

In [34]:
# On definit les labels :

train_labels = np.array([0] * 400 + [1] * 400)

validation_labels = np.array([0] * 96 + [1] * 96)

# On peut maintenant tester plusieurs couche fully-connected à partir de ce modele :

In [19]:
# Premier modele : 

model_top1 = Sequential()
model_top1.add(Flatten(input_shape=train_features.shape[1:]))
model_top1.add(Dense(64, activation='relu'))
model_top1.add(Dropout(0.5))
model_top1.add(Dense(1, activation='sigmoid'))


# On compile :

model_top1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# On affiche :

model_top1.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                6422592   
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 6,422,657
Trainable params: 6,422,657
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Second modele :

model_top2 = Sequential()
model_top2.add(Flatten(input_shape=train_features.shape[1:]))
model_top2.add(Dense(128, activation='relu'))
model_top2.add(Dropout(0.5))
model_top2.add(Dense(1, activation='sigmoid'))


# On compile :

model_top2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# On affiche :

model_top2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               12845184  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 12,845,313
Trainable params: 12,845,313
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Troisieme modele :

model_top3 = Sequential()
model_top3.add(Flatten(input_shape=train_features.shape[1:]))
model_top3.add(Dense(256, activation='relu'))
model_top3.add(Dropout(0.5))
model_top3.add(Dense(1, activation='sigmoid'))


# On compile :

model_top3.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# On affiche :

model_top3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               25690368  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 25,690,625
Trainable params: 25,690,625
Non-trainable params: 0
_________________________________________________________________


In [35]:
# On definit les parametres pour l'entrainement :

epochs = 10
train_samples = 802
validation_samples = 204

In [36]:
# On entraine le modèle 1:

hystory1 = model_top1.fit(train_features, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_features, validation_labels))

Train on 800 samples, validate on 192 samples
Epoch 1/10
800/800 [==============================] - 3s 4ms/sample - loss: 1.1906 - acc: 0.9312 - val_loss: 2.3839 - val_acc: 0.9635
Epoch 2/10
800/800 [==============================] - 3s 3ms/sample - loss: 0.1851 - acc: 0.9887 - val_loss: 1.6964 - val_acc: 0.9479
Epoch 3/10
800/800 [==============================] - 3s 3ms/sample - loss: 0.1757 - acc: 0.9825 - val_loss: 2.1452 - val_acc: 0.9688
Epoch 4/10
800/800 [==============================] - 3s 4ms/sample - loss: 0.2528 - acc: 0.9812 - val_loss: 1.7228 - val_acc: 0.9688
Epoch 5/10
800/800 [==============================] - 3s 4ms/sample - loss: 0.1398 - acc: 0.9875 - val_loss: 1.5176 - val_acc: 0.9635
Epoch 6/10
800/800 [==============================] - 3s 4ms/sample - loss: 0.0665 - acc: 0.9875 - val_loss: 1.9077 - val_acc: 0.9635
Epoch 7/10
800/800 [==============================] - 3s 3ms/sample - loss: 0.0340 - acc: 0.9937 - val_loss: 1.9517 - val_acc: 0.9635
Epoch 8/10
800/8

In [37]:
# On entraine le modèle 2:

hystory2 = model_top2.fit(train_features, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_features, validation_labels))

Train on 800 samples, validate on 192 samples
Epoch 1/10
800/800 [==============================] - 6s 7ms/sample - loss: 2.3580 - acc: 0.9400 - val_loss: 4.9403 - val_acc: 0.9583
Epoch 2/10
800/800 [==============================] - 5s 7ms/sample - loss: 0.2091 - acc: 0.9937 - val_loss: 4.0236 - val_acc: 0.9635
Epoch 3/10
800/800 [==============================] - 5s 7ms/sample - loss: 0.3316 - acc: 0.9837 - val_loss: 5.2328 - val_acc: 0.9583
Epoch 4/10
800/800 [==============================] - 5s 6ms/sample - loss: 0.4307 - acc: 0.9850 - val_loss: 4.3219 - val_acc: 0.9635
Epoch 5/10
800/800 [==============================] - 5s 6ms/sample - loss: 0.2095 - acc: 0.9887 - val_loss: 3.9515 - val_acc: 0.9635
Epoch 6/10
800/800 [==============================] - 6s 7ms/sample - loss: 0.1754 - acc: 0.9850 - val_loss: 4.1000 - val_acc: 0.9635
Epoch 7/10
800/800 [==============================] - 5s 7ms/sample - loss: 0.1382 - acc: 0.9937 - val_loss: 3.9078 - val_acc: 0.9583
Epoch 8/10
800/8

In [38]:
# On entraine le modèle 3:

hystory3 = model_top3.fit(train_features, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_features, validation_labels))

Train on 800 samples, validate on 192 samples
Epoch 1/10
800/800 [==============================] - 10s 13ms/sample - loss: 2.1112 - acc: 0.9450 - val_loss: 6.7507 - val_acc: 0.9635
Epoch 2/10
800/800 [==============================] - 10s 13ms/sample - loss: 1.6142 - acc: 0.9700 - val_loss: 6.4745 - val_acc: 0.9688
Epoch 3/10
800/800 [==============================] - 10s 12ms/sample - loss: 0.2191 - acc: 0.9950 - val_loss: 7.6351 - val_acc: 0.9323
Epoch 4/10
800/800 [==============================] - 9s 11ms/sample - loss: 0.3828 - acc: 0.9862 - val_loss: 6.5783 - val_acc: 0.9688
Epoch 5/10
800/800 [==============================] - 9s 12ms/sample - loss: 0.4963 - acc: 0.9850 - val_loss: 8.4328 - val_acc: 0.9531
Epoch 6/10
800/800 [==============================] - 10s 12ms/sample - loss: 0.3661 - acc: 0.9925 - val_loss: 6.9093 - val_acc: 0.9635
Epoch 7/10
800/800 [==============================] - 9s 12ms/sample - loss: 0.4071 - acc: 0.9862 - val_loss: 6.1754 - val_acc: 0.9531
Epoch

In [ ]:
# Visualisation de l'entrainement :



In [39]:
# On evalue le modèle 1:

model_top1.evaluate(validation_features, validation_labels,verbose=2)

# Avec les donnes de validation :

# Avec les donnes de test :

# On prend quelques exemples aleatoires :

    # on visualise, on test , on affiche :

192/192 - 0s - loss: 2.3446 - acc: 0.9688


[2.344556995813266, 0.96875]

In [40]:
# On evalue le modèle 2:

model_top2.evaluate(validation_features, validation_labels,verbose=2)

192/192 - 0s - loss: 4.1675 - acc: 0.9635


[4.167477752688397, 0.9635417]

In [41]:
# On evalue le modèle 3:

model_top3.evaluate(validation_features, validation_labels,verbose=2)

192/192 - 0s - loss: 6.8676 - acc: 0.9479


[6.867551585038503, 0.9479167]

In [ ]:
# Data augmentation :

# On reproduit le processus puis on compare :